<a href="https://colab.research.google.com/github/Justus-coded/Competitions/blob/master/DSN%20AI%20Bootcamp%20Qualification%20Hackathon/DSN%20AI%20Boot_camp%20Qualification1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
 
%matplotlib inline
 
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train = pd.read_csv('/content/drive/My Drive/DSN AI Bootcamp Qualification Hackathon/Train.csv')
test  = pd.read_csv('/content/drive/My Drive/DSN AI Bootcamp Qualification Hackathon/Test.csv')

In [5]:
dict_ = {'charge': 1,
         'lending': 0}
 
target_map = {
    'yes': 1,
    'no': 0}
 
train.default_status.replace(target_map, inplace = True)
 
train['form_field47'].replace(dict_, inplace = True)
test['form_field47'].replace(dict_, inplace = True)

In [7]:
pip install catboost

     |████████████████████████████████| 66.1MB 118kB/s 


In [8]:
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

In [9]:
features = train.select_dtypes(exclude = object).columns.drop(['default_status'])

In [10]:
# features

In [11]:
for col in features:
    train[col].fillna(-999, inplace = True)
    test[col].fillna(-999, inplace = True)

In [12]:
X = train[features]
y = train['default_status']

In [13]:
from sklearn.model_selection import StratifiedKFold

In [14]:
def metric(y, pred):
    return roc_auc_score(y, pred, labels=[0, 1])

In [15]:
# Specify number of folds
n_skf = 5
kf = StratifiedKFold(n_skf)
 
 
seed = 2020
 
## Manually tweak the hyperparameter, learning rate and n_estimators especially
params = {
    'n_estimators': 4000,
    'learning_rate': 0.005,
    'objective': 'CrossEntropy',
      'eval_metric':'AUC',
    'random_seed': seed,
    'early_stopping_rounds': 200,
    'use_best_model': True,
}

In [16]:
score_list = []
score = 0
test_oofs = []
 
for i, (tr_idx, vr_idx) in enumerate(kf.split(X, y)):
    
    xtrain, ytrain = X.loc[tr_idx, features], y.loc[tr_idx]
    xval, yval = X.loc[vr_idx, features], y.loc[vr_idx]
 
    model = CatBoostClassifier(**params, task_type='GPU')
    model.fit(xtrain, ytrain, eval_set=[(xval,yval)], verbose=100)
    
    p = model.predict_proba(xval)[:, 1]
    sc = metric(yval, p)
    score_list.append(sc)
    score += sc/n_skf
    
    pred = model.predict_proba(test[features])[:, 1]
    test_oofs.append(pred)
 
    print('Fold {} : {}'.format(i, sc))
 
print()
print()
print('Avg log : ', score)

0:	learn: 0.7882729	test: 0.7847513	best: 0.7847513 (0)	total: 97.3ms	remaining: 6m 29s
100:	learn: 0.8244553	test: 0.8209373	best: 0.8209609 (99)	total: 5.13s	remaining: 3m 18s
200:	learn: 0.8290535	test: 0.8245735	best: 0.8245735 (200)	total: 10.1s	remaining: 3m 11s
300:	learn: 0.8323058	test: 0.8273006	best: 0.8273006 (300)	total: 15s	remaining: 3m 3s
400:	learn: 0.8348172	test: 0.8292417	best: 0.8292417 (400)	total: 19.8s	remaining: 2m 58s
500:	learn: 0.8367054	test: 0.8305719	best: 0.8305719 (500)	total: 24.7s	remaining: 2m 52s
600:	learn: 0.8381771	test: 0.8314818	best: 0.8314818 (600)	total: 29.6s	remaining: 2m 47s
700:	learn: 0.8394447	test: 0.8322657	best: 0.8322657 (700)	total: 34.4s	remaining: 2m 41s
800:	learn: 0.8405741	test: 0.8328995	best: 0.8328999 (793)	total: 39.3s	remaining: 2m 36s
900:	learn: 0.8414508	test: 0.8333935	best: 0.8333935 (900)	total: 44.1s	remaining: 2m 31s
1000:	learn: 0.8422777	test: 0.8338075	best: 0.8338075 (1000)	total: 49s	remaining: 2m 26s
1100:	

In [17]:
f"{n_skf} fold CV, score: {score}" #Increase in local cv score should probably lead to increase in Leaderboard score

'5 fold CV, score: 0.8396396585139527'

### Since we are using cross validation, We'll have `n` different predictions (where n is the number of fold)

In [18]:
oof_prediction = pd.DataFrame(test_oofs).T

In [19]:
oof_prediction.columns = ['fold_'+ str(i) for i in range(1, n_skf + 1)]

In [20]:
oof_prediction.head()

,fold_1,fold_2,fold_3,fold_4,fold_5
0,0.326980,0.331161,0.347427,0.328904,0.329583
1,0.427933,0.383180,0.435164,0.341108,0.392141
2,0.335028,0.385249,0.404755,0.396653,0.350165
3,0.713925,0.727259,0.716759,0.734721,0.708495
4,0.215448,0.194006,0.195052,0.176560,0.179936


In [21]:
sub = pd.read_csv('/content/drive/My Drive/DSN AI Bootcamp Qualification Hackathon/SampleSubmission.csv')

In [22]:
### We take the mean of all 5 predictions to generate submission

In [23]:
sub['default_status'] = np.mean(test_oofs, axis = 0)

In [24]:
sub.to_csv('submit2.csv', index = False)

### Suggestions:
- Base your feature selection on the local cv score i.e When you add, remove or engineer feature check `score` (cell 17)
- Increase number of fold to 10, 15. See how score imporve on leaderboard.
- Try differnt value of learning rate. I'll advice anything < 0.01
- Create as many features as possible.
- Find code for Xgboost and lgbm using this method of modelling, or better still tweak this one (mind you, names of parameters may be slightly different.